# 4. Other data reshaping strategies

In this notebook, we'll cover:
- The `pivot_table()` method
- Filling null values
- Melting wide data to long data
- Building one dataframe out of a directory of files formatted the same way

First, let's import pandas.

In [ ]:
import pandas as pd

### `pivot_table()`

Sometimes, when you group and aggregate, the `groupby()` method doesn't _quite_ give you the view you need of your data, and the [`pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) method can be a more intuitive choice.

For this section, we'll be examining a dataset of foreign eel product imported to the U.S. ([source](https://www.fisheries.noaa.gov/national/sustainable-fisheries/foreign-fishery-trade-data)), which you will find in `../data/eels.csv`. Every row is one month's worth of eel product imports to the U.S. from one country.

In [ ]:
df_eels = pd.read_csv('../data/eels.csv')

In [ ]:
df_eels.head()

Let's take a look at import volume by country by year. If we were making this pivot table in Excel, we would drag `country` to Rows, `kilos` to Values and `year` to Columns. But we're gonna do it in pandas! We need to hand the `pivot_table()` method four things:
- The data frame you're pivoting (`df_eels`)
- The `index` column -- what to group your data by (`index='country'`)
- The `columns` column -- the second grouping factor (`columns='year'`)
- The `values` column -- what column are we doing math on? (`values='kilos'`)
- The `aggfunc` -- what function to use to aggregate the data; the default is to use an average, but we'll use Python's built-in `sum` function

In [ ]:
eels_pivoted = pd.pivot_table(df_eels,
                              index='country',
                              columns='year',
                              values='kilos',
                              aggfunc=sum)

In [ ]:
eels_pivoted

### Filling null values

Right now, if a country didn't have any shipments of these eel products in a given year, the values are `NaN`. To fill those values with zeroes instead, which might make more sense in context, you can use the [`fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) method:

In [ ]:
eels_pivoted_clean = eels_pivoted.fillna(0)

In [ ]:
eels_pivoted_clean.head()

### ✍️ Your turn

In the cells below:
- Sort the `eels_pivoted_clean` dataframe descending on the `2017` column
- Show the sum of all eel imports from 2010

### Melt wide data to long

Sometimes, you have data that's hard to work with because it's [wide instead of long](https://en.wikipedia.org/wiki/Wide_and_narrow_data). You can use the pandas [`melt()`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) method to "un-pivot" wide data into tidy data.

For this example, we're going to load in a CSV of African doctor emigration:

In [ ]:
df_doctors = pd.read_csv('../data/africa-physician-emigration.csv')

In [ ]:
df_doctors.head()

This data would be much easier to work with if every row were the sending country, the receiving country and the number of doctors. According to [the documentation](https://pandas.pydata.org/docs/reference/api/pandas.melt.html), we need to hand this method a couple of keyword arguments:
- `id_vars`: The column, or column, that will be the "identifying variable" that won't change -- in this case, `Sending country`
- `value_vars`: A column name, or a list of column names, with the values we want to melt -- in this case, everything but the identifying column (more on this in a minute)
- `var_name`: What should we call the resulting column with the variable names in it? In this case, `Receiving country`
- `value_name`: What should we call the resulting column with the values in it? In this case, `Number of doctors`

In [ ]:
df_doctors.melt(id_vars='Sending country',
                value_vars=['UK', 'USA', 'France', 'Canada', 'Australia', 'Portugal', 'Spain', 'Belgium', 'So. Africa'],
                var_name='Receiving country',
                value_name='Number of doctors')

One thing -- having to list out the names of all the columns for the `value_vars` keyword argument was kind of tedious, so here's a more efficient way to do that: by accessing the `columns` attribute of the dataframe and using list indexing to grab just the ones we want:

In [ ]:
df_doctors.columns

In [ ]:
# what do we get if we leave off the first one?
df_doctors.columns[1:]

Bingo! Let's try that again:

In [ ]:
df_doctors.melt(id_vars='Sending country',
                value_vars=df_doctors.columns[1:],
                var_name='Receiving country',
                value_name='Number of doctors')

### ✍️ Your turn

In the cells below:
- Re-run the code from the cell above but assign the new dataframe to a variable
- Filter that dataframe to show only records where the receiving country is the USA
- Filter that dataframe to show only records where the number of physicians is greater than 100
- Sort the data descending by the number of doctors emigrating -- what's the largest outflow?

### Building a dataframe from a directory of files

Sometimes, rather than one file you need to load, you have a directory of files with the same format but different data. Let's talk about a strategy for reading them all into a single dataframe -- the data for this exercise comes from [this wonderful data-driven story from 2019 by C.K. Hickey in _Foreign Policy_](https://foreignpolicy.com/all-the-presidents-meals-state-dinners-white-house-infographic/) on state dinner menus for U.S. presidents (thank you, C.K.!) and can be found in the `../data/state-dinners/` directory.

Our strategy:
- Get a list of these files using [the `glob` module](https://docs.python.org/3/library/glob.html) from the standard library
- Use a fun Python data structure called a ["list comprehension"](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) in conjunction with the pandas methods `read_csv()` (which we've seen before) and [`concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) (which we have not)

First, we need to import `glob` before we can use it. (FWIW: The customary thing to do is drop all your imports at the top of your script.)

In [ ]:
import glob

Get a list of the files using wildcards:

In [ ]:
sd_files = glob.glob('../data/state-dinners/*.csv')

In [ ]:
sd_files

In human language: Go to the `glob` module we just imported and use its `glob` object to get a list of files based on the path and filename wildcards we hand it.

Now let's talk for a sec about list comprehensions. Let's say you had a list of items that you wanted to _do_ something to -- some math, some filtering, some reading into dataframes. (We're about to do this last one!) One of the main uses for list comprehensions is effeciently "saving" the results of this operation to a new variable.

Here's a simple example -- let's say we had the following list of numbers:

In [ ]:
number_list = [1, 2, 3, 4, 5, 6]

... and we want to end up with a list of numbers that is each of these numbers multiplied by 10. We could do something like this:

In [ ]:
new_list = []
for x in number_list:
    new_list.append(x*10)

In [ ]:
new_list

You could achieve the same thing with a _list comprehension_ much quicker and easier:

In [ ]:
new_list_lc = [x*10 for x in number_list]

In [ ]:
new_list_lc

Here, `x` is a placeholder for each item in the list, same as the variable defined in the `for` loop.

That's basically what we're going to do here -- instead of creating an empty list, looping over each file in the `state_dinners` directory, creating a new dataframe, adding it to the list, then concatenating all those dataframes, we can do it all in one fell swoop:

In [ ]:
df_dinners = pd.concat([pd.read_csv(x) for x in sd_files])

Reading this from the inside out as a human sentence: Take each CSV file in the `state_dinners` directory, which we found earlier using the `glob` tool, and read it into a (more or less temporary) dataframe -- then take all of those dataframes and concatenate them together into one dataframe.

In [ ]:
df_dinners.head()